In [1]:
from pyspark import SparkContext
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/16 15:55:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [28]:
from operator import add

In [2]:
lista_tuplas = [('a',1),('b',2),('c',3)]

In [3]:
pair_rdd = sc.parallelize(lista_tuplas)

In [4]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3)]

In [5]:
pair_rdd2 = sc.parallelize(zip((['a','b','c']),range(1,4)))

In [6]:
pair_rdd2.collect()

[('a', 1), ('b', 2), ('c', 3)]

In [7]:
rdd_celestina = sc.textFile('LaCelestina.txt')

In [8]:
pair_rdd_celestina = rdd_celestina.map(lambda x: (x.split(" ")[0],x))

In [9]:
pair_rdd_celestina.takeSample(False, 15)

[('_rezio,', '_rezio, rezios, rezia_ **** recio, recios, recia;'),
 ('Ariadna:',
  'Ariadna: Leandro: que pago les diste? Hasta Dauid e Salomon no'),
 ('', ''),
 ('bienandanzas:',
  'bienandanzas: me pareces vn laberinto de errores.  Vn desierto'),
 ('(Refran:)', '(Refran:) "El loco por la pena es cuerdo.  Refr. con'),
 ('', ''),
 ('_esquiuo,', '_esquiuo, esquiua, esquiuas_ **** esquivo: aspero, malo'),
 ('loadas',
  'loadas e manifiestas virtudes.  Pues, o alto dios! como te podre'),
 ('_auise_', '_auise_ **** avise'),
 ('_pudistes_', '_pudistes_ **** pudisteis'),
 ('perdera', 'perdera aquella casa el nombre de Celestina: que dios aya:'),
 ('panjzo', 'panjzo e lubja e arbejas como se usa en la cibdat'),
 ('confiese',
  'confiese que le seria mejor estar en mediano estado / o en honesta'),
 ('ELIC.--o',
  'ELIC.--o hideputa el pelon e como se desasna: quien le ve yr al'),
 ('fin', 'fin de su habla. agora tengo por cierto: que es mas penoso al')]

In [10]:
rdd = sc.parallelize(range(5))

In [11]:
rdd.collect()

[0, 1, 2, 3, 4]

In [12]:
pair_rdd = rdd.keyBy(lambda x: x+1)

In [13]:
pair_rdd.collect()

[(1, 0), (2, 1), (3, 2), (4, 3), (5, 4)]

In [14]:
rdd = sc.parallelize(['a','b','c','d','e'])

In [15]:
pair_rdd = rdd.zipWithIndex()

In [16]:
pair_rdd.collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]

In [17]:
rdd.zipWithUniqueId().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]

In [18]:
rdd1 = sc.parallelize(range(5),3)

In [19]:
rdd2 = sc.parallelize(range(100,105,1),3)

In [20]:
rdd1.glom().collect()

[[0], [1, 2], [3, 4]]

In [21]:
rdd2.glom().collect()

[[100], [101, 102], [103, 104]]

In [22]:
pair_rdd = rdd1.zip(rdd2)

In [23]:
pair_rdd.collect()

[(0, 100), (1, 101), (2, 102), (3, 103), (4, 104)]

Transformaciones de agregación para pair rdd

In [24]:
# reduceByKey y foldByKey - son transformaciones y no acciones

In [25]:
tupla = [('a',1), ('b',2), ('c',3), ('a',4), ('b',5), ('c',6), ('c',7)]

In [26]:
pair_rdd = sc.parallelize(tupla)

In [27]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6), ('c', 7)]

In [29]:
rdd_reduce = pair_rdd.reduceByKey(add)

In [33]:
rdd_reduce.collect()

[('a', 5), ('b', 7), ('c', 16)]

In [31]:
rdd_fold = pair_rdd.foldByKey(0, add)

In [32]:
rdd_fold.collect()

[('a', 5), ('b', 7), ('c', 16)]

In [34]:
# groupByKey agrupa por clave, es costoso, se puede utilizar primero reduceByKey

In [35]:
rdd_group = pair_rdd.groupByKey()

In [36]:
rdd_group.collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x7f9f8c7f5390>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x7f9f8c7f42d0>),
 ('c', <pyspark.resultiterable.ResultIterable at 0x7f9f8c7bb590>)]

In [40]:
lista = [(x, list(y)) for x,y in rdd_group.collect()]

In [41]:
lista

[('a', [1, 4]), ('b', [2, 5]), ('c', [3, 6, 7])]

In [42]:
# combineByKey(createCombiner(), mergeValue(), mergeCombiners) - parecido a aggregate

In [45]:
rdd_suma_cuenta = pair_rdd.combineByKey(lambda x: (x,1),
                                        lambda x,y: (x[0]+y, x[1]+1),
                                        lambda x,y: (x[0],y[0],x[1]+y[1])
                                       )

In [46]:
rdd_suma_cuenta.collect()

[('a', (5, 2)), ('b', (7, 2)), ('c', (16, 3))]

In [47]:
rdd_media = rdd_suma_cuenta.mapValues(lambda v: (v[0])/v[1])

In [48]:
rdd_media.collect()

[('a', 2.5), ('b', 3.5), ('c', 5.333333333333333)]

In [49]:
# Transformaciones para claves o valores

In [50]:
pair_rdd.collect()

[('a', 1), ('b', 2), ('c', 3), ('a', 4), ('b', 5), ('c', 6), ('c', 7)]

In [51]:
pair_rdd.keys().collect()

['a', 'b', 'c', 'a', 'b', 'c', 'c']

In [52]:
pair_rdd.values().collect()

[1, 2, 3, 4, 5, 6, 7]

In [54]:
pair_rdd.sortByKey().collect()

[('a', 1), ('a', 4), ('b', 2), ('b', 5), ('c', 3), ('c', 6), ('c', 7)]

In [55]:
# mapValues - aplica la función sobre los valores
# flatMapValues - aplica la función sobre los valores y crea una lista simplificada

In [56]:
pair_rdd.mapValues(lambda x: (x, x*10)).collect()

[('a', (1, 10)),
 ('b', (2, 20)),
 ('c', (3, 30)),
 ('a', (4, 40)),
 ('b', (5, 50)),
 ('c', (6, 60)),
 ('c', (7, 70))]

In [58]:
pair_rdd.flatMapValues(lambda x: (x, x*10)).collect()

[('a', 1),
 ('a', 10),
 ('b', 2),
 ('b', 20),
 ('c', 3),
 ('c', 30),
 ('a', 4),
 ('a', 40),
 ('b', 5),
 ('b', 50),
 ('c', 6),
 ('c', 60),
 ('c', 7),
 ('c', 70)]

# Join

In [59]:
rdd1 = sc.parallelize([('a',1),('b',2)])

In [60]:
rdd2 = sc.parallelize([('a',4),('b',5), ('c',6)])

In [61]:
rdd1.collect()

[('a', 1), ('b', 2)]

In [62]:
rdd2.collect()

[('a', 4), ('b', 5), ('c', 6)]

In [63]:
rdd1.join(rdd2).collect() # Igual a un inner join

[('b', (2, 5)), ('a', (1, 4))]

In [64]:
rdd1.leftOuterJoin(rdd2).collect()

[('b', (2, 5)), ('a', (1, 4))]

In [65]:
rdd2.leftOuterJoin(rdd1).collect()

[('b', (5, 2)), ('c', (6, None)), ('a', (4, 1))]

In [66]:
rdd1.rightOuterJoin(rdd2).collect()

[('b', (2, 5)), ('c', (None, 6)), ('a', (1, 4))]

In [68]:
rdd2.rightOuterJoin(rdd1).collect()

[('b', (5, 2)), ('a', (4, 1))]

In [69]:
rdd1.fullOuterJoin(rdd2).collect()

[('b', (2, 5)), ('c', (None, 6)), ('a', (1, 4))]

In [70]:
rdd2.leftOuterJoin(rdd1).collect()

[('b', (5, 2)), ('c', (6, None)), ('a', (4, 1))]

In [71]:
# substractByKey()

In [74]:
rdd1.subtractByKey(rdd2).collect()

[]

In [75]:
rdd2.subtractByKey(rdd1).collect()

[('c', 6)]

In [76]:
# cogroup

In [77]:
rdd3 = rdd1.cogroup(rdd2)

In [78]:
rdd3.collect()

[('b',
  (<pyspark.resultiterable.ResultIterable at 0x7f9f7a72f810>,
   <pyspark.resultiterable.ResultIterable at 0x7f9f7a72f210>)),
 ('c',
  (<pyspark.resultiterable.ResultIterable at 0x7f9f8c7eec50>,
   <pyspark.resultiterable.ResultIterable at 0x7f9f7a72c3d0>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x7f9f7a74e2d0>,
   <pyspark.resultiterable.ResultIterable at 0x7f9f8c7f60d0>))]

In [82]:
rdd3.mapValues(lambda v: [list(l) for l in v]).collectAsMap()

{'b': [[2], [5]], 'c': [[], [6]], 'a': [[1], [4]]}

In [83]:
# collectAsMap

In [84]:
rdd1.collectAsMap()

{'a': 1, 'b': 2}

In [85]:
# countByKey

In [86]:
rdd1.countByKey()

defaultdict(int, {'a': 1, 'b': 1})

In [89]:
# lookup

In [88]:
rdd1.lookup('a')

[1]

In [90]:
rdd1.lookup('b')

[2]